In [ ]:
import subprocess
import numpy as np
from numpy import pi as π
import matplotlib.pyplot as plt

Create the starting file discretizing the circle.

In [ ]:
num_points = 64
θs = np.linspace(0, 2 * π, num_points + 1)[:-1]
xs = np.column_stack((np.cos(θs), np.sin(θs)))

initial_filename = "disk_initial.mesh"
with open(initial_filename, "w") as initial_file:
    initial_file.write("MeshVersionFormatted 2\n")
    initial_file.write("Dimension 2\n")
    initial_file.write("Vertices\n")
    initial_file.write(f"{num_points}\n")
    for x in xs:
        initial_file.write(f"  {x[0]} {x[1]} 0\n")

    initial_file.write("\nEdges\n")
    initial_file.write(f"{num_points}\n")
    for index in range(num_points):
        initial_file.write(f"  {index + 1} {(index + 1) % num_points + 1} 0\n")

    initial_file.write("End\n")

Generate an initial isotropic mesh.

In [ ]:
mmg_exec_path = "/home/daniel/programs/mmg/build/bin/mmg2d_O3"
iso_mesh_filename = "disk_isotropic.mesh"
hmax = 2 * π / num_points
args = [
    mmg_exec_path,
    "-in",
    initial_filename,
    "-out",
    iso_mesh_filename,
    "-hmax",
    str(hmax),
    "-v",
    "0",
]
subprocess.run(args)

Some code to read in .mesh files.

In [ ]:
def read_mesh_file(mesh_file):
    lines = mesh_file.readlines()
    line_number = ["Vertices" in line for line in lines].index(True) + 1
    num_vertices = int(lines[line_number])
    points = np.zeros((num_vertices, 2))
    vertex_lines = lines[line_number + 1: line_number + num_vertices + 1]
    for index, line in enumerate(vertex_lines):
        points[index, :] = [float(val) for val in line.split(" ")[:2]]

    line_number = ["Triangles" in line for line in lines].index(True) + 1
    num_triangles = int(lines[line_number])
    triangles = np.zeros((num_triangles, 3), dtype=int)
    triangle_lines = lines[line_number + 1: line_number + num_triangles + 1]
    for index, line in enumerate(triangle_lines):
        triangles[index, :] = [int(val) for val in line.split(" ")[:3]]

    return points, triangles - 1

Read in the isotropic mesh and plot it.

In [ ]:
with open(iso_mesh_filename, "r") as iso_mesh_file:
    points, triangles = read_mesh_file(iso_mesh_file)

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
ax.triplot(*points.T, triangles=triangles);

Create the metric.

In [ ]:
metric = np.zeros((len(points), 3))
I = np.eye(2)
r_1 = 0.2
r_2 = 0.01
ϵ = r_1
for index, x in enumerate(points):
    M = I / r_1**2 + np.outer(x, x) / (r_2 **2 * (np.dot(x, x) + ϵ**2))
    metric[index, :] = (M[0, 0], M[0, 1], M[1, 1])

Write the metric to a .sol file.

In [ ]:
metric_filename = "disk_metric.sol"
with open(metric_filename, "w") as metric_file:
    metric_file.write("MeshVersionFormatted 2\n")
    metric_file.write("Dimension 2\n")
    metric_file.write("SolAtVertices\n")
    metric_file.write(f"{len(points)}\n") 
    metric_file.write("1 3\n")
    for m in metric:
        metric_file.write(f"{m[0]} {m[1]} {m[2]}\n")
    metric_file.write("End")

In [ ]:
aniso_mesh_filename = "disk_anisotropic.mesh"
args = [
    mmg_exec_path,
    "-in",
    iso_mesh_filename,
    "-met",
    metric_filename,
    "-out",
    aniso_mesh_filename,
    "-v",
    "0",
]
subprocess.run(args)

In [ ]:
with open(aniso_mesh_filename, "r") as aniso_mesh_file:
    apoints, atriangles = read_mesh_file(aniso_mesh_file)

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
plt.triplot(*apoints.T, triangles=atriangles, linewidth=0.5);